# Archive at IRIS SPUD EMTF

With FAIR data principles and requirements for data to be public archiving has become another task for publication.  MT transfer functions can be archived at the [IRIS SPUD EMTF](http://www.ds.iris.edu/spud/emtf), which provides a web service to query for available data.  Anna Kelbert is the gatekeeper to archive transfer functions and early on was tasked with developing an XML standard for MT transfer functions.  That format (EMTFXML) is published [here](https://doi.org/10.1190/geo2018-0679.1).  XML can be complicated to read and format correctly especially of there is not a schema to validate with.  Luckily MT-Metadata has tools to convert transfer functions to the EMTFXML format.  Alternatively you may use Anna Kelbert's Fortran codes found [here](https://seiscode.iris.washington.edu/projects/emtf-fcu.).

Below are a few examples of converting transfer functions to the EMTFXML format.

## EDI to XML

Probably the most common format of MT transfer functions is the EDI format.  This was developed in the 1980's and has been a staple since due to its flexibility and readability.  However, the flexibility lends itself to be difficult to read in a standard way.  MT-Metadata supports various flavors of EDI files, probably not all so if you find one that doesn't read in properly raise an issue [here](https://github.com/kujaku11/mt_metadata/issues) with a label `transfer function`.

In [1]:
from mt_metadata import TF_EDI_RHO_ONLY
from mt_metadata.transfer_functions import TF

### Read in the Transfer Function File

First, read in the transfer function file.  Here we are reading it into the generic `TF` object.  The `TF` object contains similar metadata to time series data with the addition of a few things like how the transfer function was created.

The file we are reading is bare-bones and only contains resistivity and phase data for the off-diagonal components.  Therefore, we are going to need to add a lot of metadata to make it compliant with EMTFXML.

In [2]:
tf_object = TF(TF_EDI_RHO_ONLY)
tf_object.read()

2023-03-16 15:15:35,386 [line 379] mt_metadata.utils.mttime.MTime.from_str - ERROR: String does not contain a date: Input must be a valid datetime string, see https://docs.python.org/3.8/library/datetime.html


At this point we can do one of two things, 

1. Fill in metadata into the `TF` object then write (maybe more intuitive if you have the time series metadata)
2. Convert to an `EMTFXML` object and fill in metadata directly (maybe faster if you are just filling in metadata from scratch)

### Fill in Metadata

Before you start filling out fields, you should probably know what fields are there and what they mean.  Lets have a look at the fields

In [3]:
xml_object = tf_object.to_emtfxml()

#### Sections of an EMTFXML

Here are the key sections in an EMTFXML file, in order.  Note the names are put into lower case for easier typing, in the XML file these will be Capital Case.  `product_id` -> `ProductId`

In [4]:
for section in xml_object.element_keys:
    print(section)

description
product_id
sub_type
notes
tags
external_url
primary_data
attachment
provenance
copyright
site
field_notes
processing_info
statistical_estimates
data_types
site_layout
data
period_range


#### Getting help with attributes of each section

In [5]:
help(xml_object.copyright)

Help on Copyright in module mt_metadata.transfer_functions.io.emtfxml.metadata.copyright object:

class Copyright(mt_metadata.base.metadata.Base)
 |  Copyright(**kwargs)
 |  
 |  +----------------------------------------------+-----------------------------------------------+----------------+
 |  | **Metadata Key**                             | **Description**                               | **Example**    |
 |  +==============================================+===============================================+================+
 |  | **release_status**                           | the release status of the data                | Unrestricted   |
 |  |                                              |                                               | release        |
 |  | Required: True                               |                                               |                |
 |  |                                              |                                               |                |

#### Alternatively using print

If you just want to know what the element looks like in XML, you can use the print statement and `to_xml` method

In [6]:
print(xml_object.attachment.to_xml(string=True))

<?xml version="1.0" encoding="UTF-8"?>
<Attachment>
    <Filename/>
    <Description/>
</Attachment>



## 1. Description

`Description` is a few words about what is included in the file.  For MT transfer functions this will almost always be `Magnetotelluric Transfer Functions`, which is the default value. 

In [7]:
xml_object.description

'Magnetotelluric Transfer Functions'

## 2. ProductID

The Product ID provides a unique identifier to the transfer function, whilst including the survey, project, and station name. The format should be `{project}.{station_id}.{year}` for example data collected by a group at the USGS for station MT01 in 2020 would be `USGS.MT01.2020`.  

In [8]:
xml_object.product_id

'None.s08.2020'

In [9]:
xml_object.product_id = "USGS.MT01.2020"

## 3. SubType

`SubType` provides a secondary keyword for the what the file includes.  This will almost always be `MT_TF`, which is the default value

In [10]:
xml_object.sub_type = "MT_TF"

## 4. Notes

`Notes` allows for any free text notes about the file. 

In [11]:
xml_object.notes = "This is an example note"

## 5. Tags

`Tags` provides tags for what type of transfer functions are included in the file.  Options are 

- impedance
- tipper

Note that tags are automatically updated when `write` is called depending on the type of data the `EMTFXML` object contains.

In [12]:
xml_object.tags

'impedance'

## 6. ExternalUrl

`ExternalUrl` describes a URL that is not contained within the IRIS archive, or a URL that the data is related to that is not within the IRIS archive.  For example if the time series is archived in a different place a link to that URL would be placed here. 

It contains attributes

- **description**: A description of where and what the URL points to
- **url**: the actual URL link

In [17]:
xml_object.external_url

{
    "external_url": {
        "description": null,
        "url": null
    }
}

In [18]:
xml_object.external_url.description = "This is a fake link to non existing time series data."
xml_object.external_url.url = "fake.data.test"

## 7. Primary Data

Primary data describes an image of the transfer function that is archived along side the transfer function. It has a single attribute

- **filename**: file name of the image file 
  - *should be a .png file for easier storage and viewing*
  - *should be named {station_id}.png*

In [19]:
xml_object.primary_data.filename = "s08.png"

## 8. Attachment

Describes any attachments archived with the XML file.  For example it is good practice to archive the original file along with the XML. In this example it would be the original EDI file. Attachment has two attributes



In [20]:
xml_object.attachment

{
    "attachment": {
        "description": null,
        "filename": null
    }
}